In [1]:
import numpy as np

### Load Dataset

In [2]:
all_data = np.load('train_dev_test.npz')

In [3]:
train_encoder_output = all_data['train_encoder_output']
train_decoder_input = all_data['train_decoder_input']
train_decoder_target = all_data['train_decoder_target'][:,1:,:]
validation_encoder_output = all_data['validation_encoder_output']
validation_decoder_input = all_data['validation_decoder_input']
validation_decoder_target = all_data['validation_decoder_target'][:,1:,:]
test_encoder_output = all_data['test_encoder_output']
test_decoder_input = all_data['test_decoder_input']
test_decoder_target = all_data['test_decoder_target'][:,1:,:]

In [4]:
print("Train Encoder Output", train_encoder_output.shape)
print("Train Decoder Input", train_decoder_input.shape)
print("Train Decoder Target", train_decoder_target.shape)

Train Encoder Output (30000, 512)
Train Decoder Input (30000, 38)
Train Decoder Target (30000, 38, 2531)


In [5]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)    
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

/Users/pulkitmaloo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/pulkitmaloo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Model Architecture

In [6]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, RepeatVector, Concatenate, Merge, Masking
from keras.layers import LSTM, GRU, Embedding, TimeDistributed, Bidirectional
from keras import backend as K
from keras import optimizers
from keras.utils import plot_model

#### Parameters

In [7]:
emb_size = 300
lstm_size = 300
vocab_size = len(vocab)
max_length = train_decoder_target.shape[1]
lstm_layers = 2

lr = 0.001
dropout_rate = 0.2
batch_size = 64
n_epochs = 20

### Model

In [8]:
K.clear_session()

In [9]:
# Image -> Image embedding
image_input = Input(shape=(train_encoder_output.shape[1], ), name='image_input')
print("Image Input shape", image_input.shape)
img_emb = Dense(emb_size, activation='relu', name='img_embedding')(image_input)
print(img_emb.shape)

Image Input shape (?, 512)
(?, 300)


In [10]:
# Sentence to Word embedding
caption_inputs = Input(shape=(None, ), name='caption_input')
print("Caption Input Shape", caption_inputs.shape)
emb_layer = Embedding(input_dim=vocab_size, output_dim=emb_size, name='Embedding')
word_emb = emb_layer(caption_inputs)
print(word_emb.shape)

Caption Input Shape (?, ?)
(?, ?, 300)


In [11]:
decoder_cell = LSTM(lstm_size, return_sequences=True, return_state=True, name='decoder')
encoder_states = [img_emb, img_emb]
decoder_out, _, _ = decoder_cell(word_emb, initial_state = encoder_states)

In [12]:
output_layer = TimeDistributed(Dense(vocab_size, activation='softmax'))
decoder_out = output_layer(decoder_out)
print(decoder_out.shape)

(?, ?, 2531)


In [13]:
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1e-6)

In [14]:
model = Model(inputs=[image_input,caption_inputs], outputs=[decoder_out])

model.compile(optimizer=rmsprop,
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [15]:
model.summary()
plot_model(model, to_file='model2.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
caption_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
image_input (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding (Embedding)           (None, None, 300)    759300      caption_input[0][0]              
__________________________________________________________________________________________________
img_embedding (Dense)           (None, 300)          153900      image_input[0][0]                
__________________________________________________________________________________________________
decoder (L

In [16]:
model.fit([train_encoder_output[:10,:], train_decoder_input[:10,:]], [train_decoder_target[:10,:,:]], 
#           validation_data=([validation_encoder_output, validation_decoder_input], [validation_decoder_target]),
           epochs=n_epochs, batch_size=batch_size, verbose=1)

Epoch 1/20
10/10 [==============================] - 1s 142ms/step - loss: 2.1776 - acc: 0.0000e+00
Epoch 2/20
10/10 [==============================] - 0s 42ms/step - loss: 2.0515 - acc: 0.0500
Epoch 3/20
10/10 [==============================] - 0s 39ms/step - loss: 1.8687 - acc: 0.0500
Epoch 4/20
10/10 [==============================] - 1s 59ms/step - loss: 1.6067 - acc: 0.0500
Epoch 5/20
10/10 [==============================] - 0s 37ms/step - loss: 1.2925 - acc: 0.0421
Epoch 6/20
10/10 [==============================] - 0s 40ms/step - loss: 1.0524 - acc: 0.0447
Epoch 7/20
10/10 [==============================] - 0s 40ms/step - loss: 0.9584 - acc: 0.0316
Epoch 8/20
10/10 [==============================] - 0s 38ms/step - loss: 0.9204 - acc: 0.0474
Epoch 9/20
10/10 [==============================] - 0s 38ms/step - loss: 0.8935 - acc: 0.0447
Epoch 10/20
10/10 [==============================] - 0s 37ms/step - loss: 0.8712 - acc: 0.0632
Epoch 11/20
10/10 [==============================] - 0

## Inference

In [53]:
encoder_model = Model(image_input, encoder_states)

/Users/pulkitmaloo/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:1539: UserWarning: The list of outputs passed to the model is redundant. All outputs should only appear once. Found: [<tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>]
  ' Found: ' + str(self.outputs))


In [54]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 512)               0         
_________________________________________________________________
img_embedding (Dense)        (None, 300)               153900    
Total params: 153,900
Trainable params: 153,900
Non-trainable params: 0
_________________________________________________________________


In [55]:
decoder_state_input_h = Input(shape=(lstm_size, ))
decoder_state_input_c = Input(shape=(lstm_size, ))

In [56]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_cell(word_emb, initial_state=decoder_states_inputs)

In [57]:
decoder_states = [state_h, state_c]
decoder_outputs = output_layer(decoder_outputs)
decoder_model = Model([caption_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [58]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
caption_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding (Embedding)           (None, None, 300)    759300      caption_input[0][0]              
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
decoder (L

In [59]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [102]:
def generate_seq(img_input):
    if img_input.shape != (1, 512):
        img_input = img_input.reshape(1, 512)
    
    assert(img_input.shape == (1, 512))
    stop_condition = False
    decoded_sentence = []
    target_seq = np.array([token2idx['<bos>']]).reshape(1, 1)
    states_value = encoder_model.predict(img_input)

    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = int(np.argmax(output_tokens[0, -1, :]))

        sampled_char = idx2token[sampled_token_index]

        decoded_sentence += [sampled_char]

        if (sampled_char == '<eos>' or len(decoded_sentence) > 30):
            stop_condition = True

        target_seq = np.array([sampled_token_index]).reshape(1, 1)

        states_value = [h, c]

    return ' '.join(decoded_sentence)

In [103]:
generate_seq(train_encoder_output[1, :])

'two dogs dog dog <eos>'